In [1]:
! pip3 install -qU langchain-upstage pypdf

In [6]:
! pip3 install langchain_community

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/2.0 MB 6.9 MB/s eta 0:00:01
   ------- -------------------------------- 0.4/2.0 MB 4.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.6/2.0 MB 4.8 MB/s eta 0:00:01
   ------------- -------------------------- 0.7/2.0 MB 4.2 MB/s eta 0:00:01
   ----------------- ---------------------- 0.9/2.0 MB 3.9 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.0 MB 4.3 MB/s eta 0:00:01
   --------------------- ------------------ 1.1/2.0 MB 3.4 MB/s eta 0:00:01
   ------------------------ --------------- 1.2/2.0 MB 3.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.4/2.0 MB 3.5 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 3.5 MB/s eta 0:00:01
   ----------------

In [7]:

%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
import warnings

warnings.filterwarnings("ignore")

![SolarSample](figures/solar_sample.png)

In [9]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("pdfs/solar_sample.pdf")
docs = loader.load()  # or layzer.lazy_load()
print(docs[0].page_content[:1000]) # 표때문에 error. pdf에 text만 있는 건 아니니.

Ignoring wrong pointing object 8 0 (offset 0)


Model Size Type H6 (Avg.) ARC HellaSwag MMLU TruthfulQA Winogrande GSM8KSOLAR 10.7B-Instruct⇠11BAlignment-tuned74.2071.0888.1666.2171.4383.5864.75Qwen 72B⇠72B Pretrained 73.60 65.19 85.9477.3760.19 82.4870.43Mixtral 8x7B-Instruct-v0.1⇠47B Instruction-tuned 72.62 70.22 87.63 71.16 64.58 81.37 60.73Yi 34B-200K⇠34B Pretrained 70.81 65.36 85.58 76.06 53.64 82.56 61.64Yi 34B⇠34B Pretrained 69.42 64.59 85.69 76.35 56.23 83.03 50.64Mixtral 8x7B-v0.1⇠47B Pretrained 68.42 66.04 86.49 71.82 46.78 81.93 57.47Llama 2 70B⇠70B Pretrained 67.87 67.32 87.33 69.83 44.92 83.74 54.06Falcon 180B⇠180B Pretrained 67.85 69.4588.8670.50 45.4786.9045.94SOLAR 10.7B⇠11BPretrained66.0461.9584.6065.4845.0483.6655.50Qwen 14B⇠14B Pretrained 65.86 58.28 83.99 67.70 49.43 76.80 58.98Mistral 7B-Instruct-v0.2⇠7B Instruction-tuned 65.71 63.14 84.88 60.78 68.26 77.19 40.03Yi 34B-Chat⇠34B Instruction-tuned 65.32 65.44 84.16 74.90 55.37 80.11 31.92Mistral 7B⇠7B Pretrained 60.97 59.98 83.31 64.16 42.15 78.37 37.83Table 2: Ev

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [11]:
chain.invoke({"question": "Explain Table 2?", "Context": docs})

'Table 2 in the context provided is the evaluation results in the Open LLM Leaderboard for SOLAR 10.7B and SOLAR 10.7B-Instruct along with other top-performing models. The table reports the scores for six tasks mentioned in Sec. 4.1, along with the H6 score (average of six tasks). The table also reports the size of the models in units of billions of parameters and the type of training stage of the model, which is chosen from {Pretrained, Instruction-tuned, Alignment-tuned}. The best scores for H6 and the individual tasks are shown in bold. The models based on SOLAR 10.7B are colored purple.'

In [12]:
chain.invoke({"question": "What is MMLU scores of SOLAR 10.7B?", "Context": docs}) # 있는데 없대~

'The information is not present in the context.'

In [13]:
chain.invoke({"question": "What is ARC of Mistral?", "Context": docs})

'The information is not present in the context.'

# Excercise 

How can we easily get information from complicated tables for LLMs?